# Convertendo a base de dados de CSV para Parquet

- **Objetivo**: Converter um arquivo CSV para o formato Parquet, que é mais eficiente para armazenamento e consulta de grandes volumes de dados.

- **Motivação**: Atualmente é praticamente impossível trabalhar com a base de dados bruta em CSV que tem mais de 20GB.

In [ ]:
import pandas as pd

# Carrega os dados do CSV (pode demorar alguns minutos)
raw_data = pd.read_csv("DB/dependencies-1.6.0-2020-01-12.csv", dtype={9: str})

In [ ]:
# Salva os dados em Parquet (novamente, pode demorar alguns minutos)
raw_data.to_parquet("DB/dependencies-1.6.0-2020-01-12.parquet")

In [ ]:
pd.set_option("display.max_rows", 50)
display(raw_data)

## Filtrando a base de dados
- **Objetivo**: Filtrar a base de dados para manter apenas as versões mais recentes de cada dependência.

- **Motivação**: A base de dados contém muitas versões antigas de dependências, o que torna a análise mais complexa e menos eficiente. Manter apenas as versões mais recentes facilita a análise e reduz o tamanho do arquivo.

In [ ]:
import pandas as pd
from packaging import version
import numpy as np

# Carrega o arquivo Parquet
df = pd.read_parquet("DB/dependencies-1.6.0-2020-01-12.parquet")

In [ ]:
# ESSA OPERAÇÃO DEMOROU 41 MINUTOS COM A SEGUINTE CONFIGURAÇÃO:
# CPU Intel(R) Xeon(R) CPU E5-2680 v3 @ 2.50GHz
# RAM 256 GB

def parse_version_safe(version_str):
    """Converte string de versão para objeto Version do packaging"""
    try:
        # Remove prefixos comuns e limpa a string
        clean_version = str(version_str).strip().lstrip('v')
        # Remove sufixos como '+build' ou '-alpha'
        clean_version = clean_version.split('+')[0].split('-')[0]
        return version.Version(clean_version)
    except:
        # Para versões inválidas, retorna uma versão muito baixa
        return version.Version("0.0.0")

# Aplica parsing de versão de forma vetorizada
print("Parsing versions...")
df['version_obj'] = df['Version Number'].apply(parse_version_safe)

# Agrupa por Project Name E Platform para encontrar versão máxima
# (importante porque o mesmo projeto pode existir em plataformas diferentes)
print("Finding latest versions...")
latest_versions = df.groupby(['Project Name', 'Platform'])['version_obj'].transform('max')

# Filtra mantendo apenas as versões mais recentes
print("Filtering dataframe...")
df_latest = df[df['version_obj'] == latest_versions].copy()

# Remove coluna auxiliar para economizar memória
df_latest.drop('version_obj', axis=1, inplace=True)

print(f"DataFrame original: {len(df):,} linhas")
print(f"DataFrame filtrado: {len(df_latest):,} linhas")
print(f"Redução: {((len(df) - len(df_latest)) / len(df) * 100):.1f}%")

# Salva o DataFrame filtrado em Parquet
df_latest.to_parquet("DB/dependencies-1.6.0-latest_versions.parquet", index=False)

- **Absolute latest version**: A versão mais recente de cada dependência independete da plataforma.

In [4]:
import pandas as pd
from packaging import version
import numpy as np

# Carrega o arquivo Parquet
df = pd.read_parquet("DB/dependencies-1.6.0-latest_versions.parquet")

In [6]:
# ESSA OPERAÇÃO DEMOROU 7 MINUTOS COM A SEGUINTE CONFIGURAÇÃO:
# CPU Intel(R) Xeon(R) CPU E5-2680 v3 @ 2.50GHz
# RAM 256 GB

def parse_version_safe(version_str):
    """Converte string de versão para objeto Version do packaging"""
    try:
        # Remove prefixos comuns e limpa a string
        clean_version = str(version_str).strip().lstrip('v')
        # Remove sufixos como '+build' ou '-alpha'
        clean_version = clean_version.split('+')[0].split('-')[0]
        return version.Version(clean_version)
    except:
        # Para versões inválidas, retorna uma versão muito baixa
        return version.Version("0.0.0")

# Aplica parsing de versão de forma vetorizada
print("Parsing versions...")
df['version_obj'] = df['Version Number'].apply(parse_version_safe)

# Encontra a versão ABSOLUTA mais recente para cada projeto (ignorando plataforma)
print("Finding absolute latest versions...")
latest_versions = df.groupby('Project Name')['version_obj'].transform('max')

# Filtra mantendo apenas as versões mais recentes absolutas
print("Filtering to absolute latest versions...")
df_absolute_latest = df[df['version_obj'] == latest_versions].copy()

# Remove coluna auxiliar
df_absolute_latest.drop('version_obj', axis=1, inplace=True)

print(f"DataFrame original: {len(df):,} linhas")
print(f"DataFrame com versões absolutas mais recentes: {len(df_absolute_latest):,} linhas")
print(f"Redução: {((len(df) - len(df_absolute_latest)) / len(df) * 100):.1f}%")

# Salva o DataFrame filtrado em Parquet
df_absolute_latest.to_parquet("DB/dependencies-1.6.0-absolute_latest_versions.parquet", index=False)

Parsing versions...
Finding absolute latest versions...
Filtering to absolute latest versions...
DataFrame original: 19,557,940 linhas
DataFrame com versões absolutas mais recentes: 19,253,961 linhas
Redução: 1.6%


- **Filtro de informações não relevantes**: remove linhas de informações não relevantes.

In [4]:
import pandas as pd

# Carrega arquivo parquet
df = pd.read_parquet("DB/dependencies-1.6.0-absolute_latest_versions.parquet")
# remove linhas com valores ausentes nas colunas "Project Name" e "Dependency Name"
df.dropna(subset=["Project Name", "Dependency Name"], inplace=True)
# filtra linhas desnecessárias
df = df[~df["Project Name"].str.match(r"^all-packages-\d+.*", na=False)]
df = df[~df["Project Name"].str.match(r"^cool-\d+.*", na=False)]
df = df[~df["Project Name"].str.match(r"^neat-\d+.*", na=False)]
df = df[~df["Project Name"].str.match(r"^wowdude-\d+.*", na=False)]

# Salva o DataFrame filtrado em Parquet
df.to_parquet("DB/dependencies-1.6.0-clean.parquet", index=False)

In [6]:
# remove as colunas mantendo apenas  "Project Name", "Dependency Name" e cria uma lista de dependências
df = df[["Project Name", "Dependency Name"]]
# salva o DataFrame filtrado em Parquet
df.to_parquet("DB/List-dependencies.parquet", index=False)

In [8]:
df.to_csv("DB/List-dependencies.csv", index=False)

In [ ]:
# Teste de filtro
filtro = r"^wowdude-\d+.*"
display(df[df["Project Name"].str.match(filtro, na=False)])
display(df[df["Dependency Name"].str.match(filtro, na=False)])